In [1]:
pip install --user --upgrade openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade bottleneck

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import pandas as pd

# Path to the uploaded images and Excel file
image_folder = r'C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\Panda Express images'
excel_file = r'C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\PandaExpress.xlsx'

# Load the Excel dataset
data = pd.read_excel(excel_file, sheet_name='Sheet1')

# Preprocess the Dish Name column
data['Formatted Dish Name'] = data['Dish Name'].str.replace(' ', '_').str.strip().str.lower()

# Create a mapping list
mapping = []

# Scan the image folder and parse filenames
for file in os.listdir(image_folder):
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.webp', '.gif')):
        # Extract the dish name by removing the variant and extension
        dish_name = os.path.splitext(file)[0].rsplit('_', 1)[0].lower()  # Remove last '_<Variant>'
        
        # Exact match with the Excel data
        dish_row = data[data['Formatted Dish Name'] == dish_name]
        
        # If no exact match, attempt partial matching
        if dish_row.empty:
            dish_row = data[data['Formatted Dish Name'].str.contains(dish_name)]
        
        # Append to the mapping if matched
        if not dish_row.empty:
            mapping.append({
                'Image Filename': file,
                'Dish Name': dish_row.iloc[0]['Dish Name'],
                'Calories': dish_row.iloc[0]['Calories'],
                'Category': dish_row.iloc[0]['Category']
            })
        else:
            print(f"No match found for: {file} (Parsed dish name: {dish_name})")

# Create a DataFrame from the mapping
mapping_df = pd.DataFrame(mapping)

# Save the mapping table to a CSV file
mapping_csv_path = r'C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\ImageDishMapping.csv'
mapping_df.to_csv(mapping_csv_path, index=False)

print(f"Mapping table saved to {mapping_csv_path}")

Mapping table saved to C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\ImageDishMapping.csv


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Paths to training and validation data
train_dir = r'C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\data\train'
validation_dir = r'C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\data\validation'

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Load MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Add custom classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Save the fine-tuned model
model.save('pretrained_panda_express_model.h5')

Found 25 images belonging to 5 classes.
Found 9 images belonging to 5 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


C:\Users\anamt\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.1200 - loss: 2.0487 - val_accuracy: 0.5556 - val_loss: 1.5736
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3600 - loss: 1.5882 - val_accuracy: 0.5556 - val_loss: 1.3609
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5600 - loss: 1.1763 - val_accuracy: 0.6667 - val_loss: 1.1588
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7600 - loss: 0.7954 - val_accuracy: 0.7778 - val_loss: 1.0321
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8000 - loss: 0.7199 - val_accuracy: 0.7778 - val_loss: 0.9269
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8000 - loss: 0.7487 - val_accuracy: 0.7778 - val_loss: 0.8019
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8800 - loss: 0.4155 - val_accuracy: 0.7778 - val_loss: 0.6806
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8400 - loss: 0.4866 - val_accuracy: 0.7778 - val_loss: 0.5612
Epoch 9/10
1/1 ━

In [8]:
from tensorflow.keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('pretrained_panda_express_model.h5')

# def predict_dish(image_path):
#     img = image.load_img(image_path, target_size=(224, 224))
#     img_array = image.img_to_array(img) / 255.0
#     img_array = np.expand_dims(img_array, axis=0)

#     prediction = model.predict(img_array)
#     class_idx = np.argmax(prediction)
#     return train_generator.class_indices, class_idx

def predict_dish(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_labels = {v: k for k, v in train_generator.class_indices.items()}
    return class_labels[class_idx], prediction[0][class_idx]

# Example usage:
dish_name, confidence = predict_dish(r'C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\Menu Photos\Panda Express\test0.jpg')
print(f"Predicted dish: {dish_name} with confidence {confidence:.2f}")


# print(predict_dish(r'C:\Users\anamt\OneDrive - Cal State Fullerton\RA\AI Food Project\Menu Photos\Panda Express\test0.jpg'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted dish: Super_Greens with confidence 0.89
